In [1]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da

In [33]:
cluster = LSFCluster(cores=15, memory='250GB', queue='bigmem', walltime='00:10')
cluster

In [34]:
client = Client(cluster)
client

Client Scheduler: tcp://10.10.0.13:44562 Dashboard: http://10.10.0.13:50006/status,Cluster Workers: 4 Cores: 60 Memory: 1000.00 GB


In [35]:
df = dd.demo.make_timeseries(start='2000-01-01',
                             end='2000-12-31',
                             dtypes={'x': float, 'y': float, 'id': int},
                             freq='10ms',
                             partition_freq='24h')
df

,id,x,y
npartitions=365,,,
2000-01-01,int64,float64,float64
2000-01-02,...,...,...
...,...,...,...
2000-12-30,...,...,...
2000-12-31,...,...,...


In [36]:
df.head()

,id,x,y
timestamp,,,
2000-01-01 00:00:00.000,1013,0.849512,0.443433
2000-01-01 00:00:00.010,989,0.063587,0.636414
2000-01-01 00:00:00.020,993,0.627234,0.766118
2000-01-01 00:00:00.030,1006,0.400670,-0.619518
2000-01-01 00:00:00.040,976,0.310427,0.760433


In [37]:
df = df.persist()

In [38]:
# Calculate the mean of the x and y columns for different ids
%time df.groupby('id')[['x', 'y']].mean().compute()

CPU times: user 4.13 s, sys: 311 ms, total: 4.44 s
Wall time: 11.7 s


,x,y
id,,
834,0.084535,-0.036087
839,-0.075702,-0.016139
846,-0.005721,-0.067186
850,0.006626,0.023603
853,-0.010294,-0.016119
855,-0.007222,0.019299
856,0.014822,0.005952
857,0.030466,0.026414
858,-0.020837,0.006224


In [39]:
# Calculate a rolling 1 minute window. Take the standard deviation for each windows. 
# Remove the first and last day of data. Find the index that corresponds to the maximum.
%time df.x.rolling('1min').std().loc['2000-01-02':'2000-12-30'].idxmax().compute()

CPU times: user 1min 14s, sys: 6.28 s, total: 1min 20s
Wall time: 5min 40s


Timestamp('2000-10-03 21:53:21.400000')

In [41]:
# Calculate the singular vector decomposition# Calcu 
u, s, v = da.linalg.svd(df.values + 1)
s

dask.array<getitem-d64bfb400719bde8e4fb1e82760c8eab, shape=(3,), dtype=float64, chunksize=(3,)>

In [42]:
%time s.compute()

CPU times: user 7.09 s, sys: 600 ms, total: 7.69 s
Wall time: 26.8 s


array([5.62411570e+07, 3.25189399e+04, 3.24223565e+04])